In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_8612/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_8612/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_28_7,0.989762,0.805934,0.871929,0.993728,0.042966,1.297723,0.458104,0.052094,0.569884,0.207283,1.027301,0.216107,72.294681,112.517583,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_28_3,0.989123,0.805883,0.872059,0.993758,0.045650,1.298064,0.457637,0.051845,0.591529,0.213658,1.029006,0.222754,72.173517,112.396419,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_24_0,0.980668,0.805862,0.861230,0.986058,0.081130,1.298204,0.496371,0.109203,0.835559,0.284834,1.051551,0.296960,71.023397,111.246299,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_24_6,0.982930,0.805829,0.863428,0.987801,0.071640,1.298423,0.488511,0.095554,0.790643,0.267656,1.045520,0.279051,71.272206,111.495108,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_28_0,0.988585,0.805803,0.872114,0.993723,0.047907,1.298595,0.457439,0.052133,0.608372,0.218876,1.030440,0.228194,72.077008,112.299910,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,model_1_40_8,0.900321,0.613013,0.673029,0.908601,0.418330,2.587783,1.169557,0.165411,2.072108,0.646785,0.658244,0.674320,35.742967,56.463856,"Hidden Size=[4], regularizer=0.02, learning_ra..."
380,model_1_43_0,0.889079,0.611711,0.674233,0.826991,0.465511,2.596491,1.165251,0.420932,1.752782,0.682284,0.619700,0.711330,35.529239,56.250129,"Hidden Size=[4], regularizer=0.02, learning_ra..."
381,model_1_41_1,0.905627,0.610311,0.669899,0.824010,0.396064,2.605852,1.180751,0.509071,1.979585,0.629336,0.676435,0.656128,35.852360,56.573249,"Hidden Size=[4], regularizer=0.02, learning_ra..."
382,model_1_42_6,0.898760,0.586979,0.653900,0.678190,0.424881,2.761874,1.237979,0.848327,1.719839,0.651829,0.652893,0.679578,35.711894,56.432783,"Hidden Size=[4], regularizer=0.02, learning_ra..."
